### MIP optimisation using LP relaxation

Based on the LP Bound Algorithm presented in:
'Analytics for an Online Retailer: Demand Forecasting and Price Optimization'
https://doi.org/10.1287/msom.2015.0561

In [ ]:
import numpy as np
import time
from mip import Model, xsum, maximize, BINARY, OptimizationStatus
from scipy.optimize import linprog

In [ ]:
def lp_mip_solver(demands,prices,sum_prices):

# the function takes three inputs. 
# demands: matrix of size 'n*k*j' with n the # of items, k the # of price possibilities, and j # the number of sum of prices considered   
# prices: list of 'k' possible prices 
# sum_prices: vector typically ranging from n*min(prices) to n*max(prices) 
    
    n, k, num_loops = np.shape(demands) # n corresponds to the number of products and k to the number of prices considered in the optimisation problem


    # sanity check
    assert  num_loops == len(sum_prices), 'the demands matrix last dimension is different from len(sum_prices)'
    assert  k == len(prices), 'the demands matrix middle dimension is different from len(prices)'


    # initialising empty variables
    optimum_solution = np.zeros(n*k) # optimum solution 
    revenue_prediction = 0 # optimum revenue

        

    # Constraints are recast out of the shape A*x = b. 
    # Two types of constraints are considered. The sum of the prices of a single item must be equal to 1 (for a binary variable this means that an item has only a single price!)
    A = np.array([[
        1 if j >= k*(i) and j < k*(i+1) else 0
        for j in range(k*n)
    ] for i in range(n)])


    # The second set of constraints is defined and added to A here: The sum of the prices must be equal to sum_prices[aa]
    A = np.append(A, np.tile([prices], n), axis=0)


    objective_loop, LBk = loop_k(demands, prices, sum_prices, A)

    # step 2 (LP bound Algorithm) in `Analytics for an Online Retailer: Demand Forecasting and Price Optimization`
    sum_prices_sorted = sum_prices[np.argsort(objective_loop)[::-1]]
    objective_sorted = objective_loop[np.argsort(objective_loop)[::-1]]
    demands_sorted = demands[:,:,np.argsort(objective_loop)[::-1]]
    LBk_sorted = LBk[np.argsort(objective_loop)[::-1]]

    
    # step 3 (LP bound Algorithm)
    k_hat = np.argmax(LBk_sorted)
    LB = LBk_sorted[k_hat]

    ll = 0
    flag = True
    while flag == True: # looping voer the MIP probelm while flag == True
        
        demands_submatrix = demands_sorted[:,:,ll]
        r = np.multiply(np.tile([prices], n), np.array(demands_submatrix).reshape(1, k*n)).flatten()
        b = [np.append(np.ones(n), sum_prices_sorted[ll])] # constraint vector: [1....1 sum_prices[ll]]
        m = Model() # calling the model object and initiation

        x = [m.add_var(var_type=BINARY) for i in range(k*n)] # defining the different variables: n*k variables and defining them as binary

        m.objective = maximize(xsum(r[i] * x[i] for i in range(k*n))) # objective function defined as r*x to be maximised

        for j in range(n+1): # adding the different constraints to the problem Ax = b
            m += xsum(A[j,i] * x[i] for i in range(n*k)) == b[0][j]

        status = m.optimize() # calling the solver

        if status == OptimizationStatus.OPTIMAL and m.objective_value > LB:
            k_hat = ll
            LB = m.objective_value
            revenue_prediction = m.objective_value # then we want this to be our objective
            optimum_solution = np.array([ x[aa].x for aa in range(n*k)]) #recording the solution
        
        if ll==num_loops-1:
            flag = False
        elif status == OptimizationStatus.OPTIMAL and LB >= objective_sorted[ll+1]:
            flag = False
        else:
            ll+=1
            
            
    optimal_prices = np.matmul(optimum_solution.reshape(n,k),prices) # returning the vector of optimum prices for each item

    return optimal_prices, revenue_prediction

In [ ]:
def loop_k(demands, prices, sum_prices, A):
    # function where the loop over the different values of the sum of the prices: sum_prices[aa] is executed
    
    
    n, k, num_loops = np.shape(demands) 

    
    # r is the vector giving the different price * demand combinations 
    # It is used to define the cost function to maximise: max_x( tranpose(r) * x )

    #initialisation
    objective_loop = np.zeros(len(sum_prices)) # best objective function in each solution
    LBk = np.zeros(len(sum_prices))

    for aa in range(num_loops): 
        demands_submatrix = demands[:,:,aa]
        r = np.multiply(np.tile([prices], n), np.array(demands_submatrix).reshape(1, k*n))

        b = [np.append(np.ones(n), sum_prices[aa])] # constraint vector: [1....1 sum_prices[aa]]
        
        lp_sol = linprog(-r.flatten(), A_eq = A, b_eq = b) # calling the model object and initiation
        

        if lp_sol.status == 0:
            objective_loop[aa] = -lp_sol.fun
            LBk[aa] = -lp_sol.fun - np.max( np.max(prices*demands_submatrix,axis=1) - np.min(prices*demands_submatrix,axis=1) )
    

    
    return objective_loop, LBk

In [ ]:
# collection of debugging examples

#prices = np.array([1.00, 1.50, 2.00, 2.50])
#demands = np.array([
#    [ 28, 23, 20, 13],                   
#    [ 30, 22, 16, 12],                   
#    [ 32, 26, 19, 15]]).reshape(3,4,1)
#sum_prices = np.array([1.5*3])


#prices = np.array([1.00, 1.50, 2.00])
#demands = np.zeros((2,3,2))
#demands[:,:,0] = np.array([[ 3,2,1.5],[ 3, 1.5, 2]])
#demands[:,:,1] = np.array([
#    [ 2,1.5,3],                   
#    [ 2, 3, 1.5]])
#sum_prices = np.array([3.5,4])

#prices = np.arange(30,310,10)
#demands = np.random.rand(24,np.shape(prices)[1])
#sum_prices = np.arange(30*24,24*300+110,10)

prices = np.arange(30,100,10)
sum_prices = np.arange(30*24,40*24+10,10)
demands = np.random.rand(24,len(prices),len(sum_prices))

print(len(sum_prices), len(prices))


In [ ]:
optimal_prices, revenue_prediction = lp_mip_solver(demands,prices,sum_prices)

In [ ]:
print(optimal_prices)
print(revenue_prediction)